<a href="https://colab.research.google.com/github/Magdy007/AIMusician/blob/main/GEPO_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ربط جيت هب وحوجل درايف لضمان حفظ الجلسات

ربط جيتهب

In [2]:
from google.colab import auth
auth.authenticate_user()

!git config --global user.email "your-email@example.com"
!git config --global user.name "YourGitHubUsername"


التحقق من ربط جيت هب

In [3]:
!git config --global user.email
!git config --global user.name


your-email@example.com
YourGitHubUsername


استرجاع المشروع من جيت هب

In [ ]:
!git clone https://github.com/Magdy007/AIMusician.git
%cd AIMusician


Cloning into 'AIMusician'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 17 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 319.46 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/AIMusician/AIMusician


📌 إضافة كل الملفات المعدلة والجديدة إلى قائمة الرفع:
اذا كنت تملك ... على جيتهب

In [ ]:
!git add .


معرفة حالة الملفات التي تم رفعها من جيت هب

In [ ]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


📌 حفظ التعديلات داخل Git بتعليق يصف التغيير:

In [ ]:
!git commit -m "AIMusician"


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


ربط جوجل درايف المستخدم أو التأكد من الربط لضمان حفظ العمل الموسيقي

In [1]:
import os
import time
import threading
from google.colab import drive

# التحقق مما إذا كان Google Drive مربوطًا بالفعل
if os.path.exists("/content/drive/MyDrive"):
    print("✅ Google Drive مربوط بالفعل!")
else:
    print("🔗 Google Drive غير مربوط! يرجى الضغط على الرابط التالي لربطه:")
    drive.mount('/content/drive')
    print("✅ تم ربط Google Drive بنجاح!")

# دالة لحفظ النوتبوك تلقائيًا
def auto_save():
    while True:
        time.sleep(600)  # 600 ثانية = 10 دقائق
        os.system("cp /content/*.ipynb /content/drive/MyDrive/")  # حفظ كل النوتبوك إلى Google Drive
        print("✅ تم حفظ نسخة احتياطية تلقائيًا في Google Drive!")

# تشغيل الحفظ التلقائي في Thread منفصل
thread = threading.Thread(target=auto_save, daemon=True)
thread.start()


✅ Google Drive مربوط بالفعل!


In [5]:
!pip install face_recognition


In [4]:
!pip install dlib-bin


In [4]:
import dlib
import face_recognition
import numpy as np
from PIL import Image

print("✅ تم تحميل المكتبات بنجاح!")
print("dlib.DLIB_USE_CUDA:", dlib.DLIB_USE_CUDA)


✅ تم تحميل المكتبات بنجاح!
dlib.DLIB_USE_CUDA: False


In [5]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import librosa
import face_recognition
import numpy as np
import tempfile
from io import BytesIO
from PIL import Image

# Disable CUDA to force CPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Verify dlib and face_recognition installation
try:
    import dlib
    print("dlib.DLIB_USE_CUDA:", dlib.DLIB_USE_CUDA)
except ImportError:
    print("⚠️ Warning: dlib is not installed correctly!")

# Global variables
uploaded_audio = None
uploaded_image = None
chat_output = []
chat_ui = widgets.Output()

def analyze_audio(audio_data):
    with tempfile.NamedTemporaryFile(delete=True, suffix=".wav") as tmp_file:
        tmp_file.write(audio_data)
        tmp_file.flush()
        y, sr = librosa.load(tmp_file.name, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    return duration

def analyze_image(image_data):
    image = Image.open(BytesIO(image_data))
    image_np = np.array(image)
    face_locations = face_recognition.face_locations(image_np, model="hog")
    return len(face_locations) > 0

def on_upload_audio(change):
    global uploaded_audio
    uploaded_audio = list(change['new'].values())[0]['content']
    chat_output.append("✅ Audio uploaded successfully!")
    print("Audio uploaded:", uploaded_audio is not None)
    update_ui()

def on_upload_image(change):
    global uploaded_image
    uploaded_image = list(change['new'].values())[0]['content']
    chat_output.append("✅ Image uploaded successfully!")
    print("Image uploaded:", uploaded_image is not None)
    update_ui()

def update_ui():
    with chat_ui:
        clear_output(wait=True)  # تجنب flickering
        for message in chat_output:
            print(message)
        display(upload_audio, upload_image)

        if uploaded_audio and uploaded_image and "⏳ Analyzing files..." not in chat_output:
            chat_output.append("⏳ Analyzing files...")
            print("Starting analysis...")

            # Perform analysis
            audio_duration = analyze_audio(uploaded_audio)
            face_detected = analyze_image(uploaded_image)

            if not face_detected:
                chat_output.append("❌ No face detected in the image. Please upload a clear image.")
            else:
                chat_output.append(f"✅ Analysis complete! Audio duration: {audio_duration:.2f} seconds. Face detected in the image.")
                chat_output.append("🎭 Now, select animation settings.")
                display(animation_options)

# Separate upload widgets for audio and image
upload_audio = widgets.FileUpload(accept='.mp3,.wav', multiple=False, description='Upload Audio')
upload_audio.observe(on_upload_audio, names='value')

upload_image = widgets.FileUpload(accept='.png,.jpg,.jpeg', multiple=False, description='Upload Image')
upload_image.observe(on_upload_image, names='value')

# Animation options widget
animation_options = widgets.Dropdown(
    options=['Calm', 'Expressive', 'Neutral'],
    description='Animation:',
    style={'description_width': 'initial'}
)

def start_chatbot():
    with chat_ui:
        clear_output()
        chat_output.append("👋 Welcome! Please upload an image and an audio file to get started.")
        display(upload_audio, upload_image)

# Run chatbot
start_chatbot()
display(chat_ui)


dlib.DLIB_USE_CUDA: False


Output()

Audio uploaded: True
Image uploaded: True
